In [1]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from processor.processor import Processor as p
from database.adatabase import ADatabase
from indicator.indicator import Indicator
from backtester.backtester import Backtester
from time import sleep
import requests as r
from dotenv import load_dotenv
load_dotenv()
import os
import matplotlib.pyplot as plt
from time import sleep
import numpy as np
from parameters.parameters import Parameters

In [2]:
market = ADatabase("market")
chung = ADatabase("chung")

In [3]:
market.connect()
currencies = market.retrieve("currencies")
stocks = market.retrieve("index")
market.disconnect()
crypto = False

In [4]:
if crypto == True:
    index = currencies.copy().rename(columns={"product_id":"ticker"})
    index["name"] = index["ticker"]
    index["base"] = [x.split("-")[1] for x in index["ticker"]]
    index = index[index["base"]=="USD"]
    market.connect()
    spy = market.query("crypto",{"ticker":"BTC-USD"}).rename(columns={"close":"adjclose"})[["start","adjclose","high","low","volume"]]
    spy["date"] = pd.to_datetime(spy["start"].astype(float),unit="s")
    spy["spy"] = spy["adjclose"].astype(float)
    spy["high"] = spy["high"].astype(float)
    spy["low"] = spy["low"].astype(float)
    spy["volume"] = spy["volume"].astype(float)
    market.disconnect()
else:
    index = stocks.copy()
    market.connect()
    benchmark = p.column_date_processing(market.query("prices",{"ticker":"JPM"}))
    benchmark["benchmark"] = benchmark["adjclose"]
    market.disconnect()

In [5]:
chung.connect()
product = chung.retrieve("sim")
chung.disconnect()
product = p.column_date_processing(product).sort_values("date")

In [6]:
weekly_dates = [x for x in product["date"].unique() if x >= datetime(2008,1,1)]

In [7]:
parameters = Parameters.generate_bruteforce_parameters()
print(len(parameters))

90


In [8]:
chung.connect()
chung.drop("spec_adaptive_trades")
remodel_weeks = 52
performance_metric = "pnl"
for date in tqdm(weekly_dates[:-1:52]):
    train_data = product[(product["date"]<date) & (product["date"]>=date-timedelta(days=365.25*4))].sort_values("date")
    test_data = product[(product["date"]>=date) & (product["date"]<=date+timedelta(days=remodel_weeks*7))].sort_values("date")
    
    best_outperformance = -float('inf')
    best_parameters = None
    for parameter in parameters:
        try:
            results, trades,portfolio = Backtester.run_strategy(train_data.copy(), parameter.copy(),benchmark.copy())  # Ensure results are calculated
            if results[performance_metric] > best_outperformance:
                best_outperformance = results[performance_metric]
                best_parameters = parameter.copy()
        except Exception as e:
            print(str(e))
            continue
    results,trades, portfolio = Backtester.run_strategy(test_data, best_parameters.copy(),benchmark.copy())
    chung.store("spec_adaptive_trades",trades)
chung.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [06:35<00:00, 14.63s/it]
